In [1]:
#imports
import pandas as pd
import numpy as np

In [ ]:
''' Tratamento dos dados:
    - Ratings possui as interações usuário-item
    - Targets possui os dados para qual precisamos prever os resultados
'''

# Ler o arquivo CSV original para um DataFrame
df_ratings_original = pd.read_csv('ratings.csv', sep=',', header=0, names=["UserId:ItemId", "Rating"])
df_ratings_original.head()

# Dividir a coluna "UserId:ItemId" em duas colunas separadas usando ":"
df_ratings_split = df_ratings_original["UserId:ItemId"].str.split(":", expand=True)
df_ratings_split.columns = ["UserId", "ItemId"]

# Concatenar as colunas divididas com a coluna "Rating" em um novo DataFrame
df_ratings = pd.concat([df_ratings_split, df_ratings_original["Rating"]], axis=1)


# Ler o arquivo CSV targets.csv para df_targets_original
df_targets = pd.read_csv('targets.csv', sep=':', header=0, names=["UserId","ItemId"])

#Adicionar a coluna "Rating" ao DataFrame df_targets_original
df_targets["Rating"] = 0

In [ ]:
#Para um usuário/item no dataframe targets, calcula o rating previsto usando a função SGD

def predict_rating(user, item, P, Q, user_index, item_index):
    if user in user_index and item in item_index:
        result = np.dot(P[user_index[user]], Q[item_index[item]].T)
        if result > 5:
            result = 5
        elif result < 1:
            result = 1
        return result

In [ ]:
#Para cada usuário/item no dataframe targets, calcula o rating previsto usando a função SGD
def predict_all_ratings(df_targets, P, Q, user_index, item_index):
    df_targets["Rating"] = df_targets.apply(lambda x: predict_rating(x["UserId"], x["ItemId"], P, Q, user_index, item_index), axis=1)
    return df_targets

In [ ]:
''' SGD (Stochastic gradient descent)
    - S é o dataframe que armazena os ratings de treino
    - k é o tamanho dos vetores latentes 
    - alpha é a taxa de aprendizado
    - lamb é a variável de regularização
    - epochs é o número de épocas usadas no algoritmo
'''


def SGD(S, k=18, alpha=0.01, lamb=0.02, epochs=10):

    #Contar o numero de usuarios e itens unicos
    n_users = S["UserId"].nunique()
    n_items = S["ItemId"].nunique()

    #Criar um dicionario para mapear os usuarios e itens
    user_index = {original: new for new, original in enumerate(S["UserId"].unique())}
    item_index = {original: new for new, original in enumerate(S["ItemId"].unique())}

    np.random.seed(15)

    # Inicializar os vetores de usuário e item com valores aleatórios
    P = np.random.rand(n_users, k)
    Q = np.random.rand(n_items, k)

    SList = S[["UserId", "ItemId", "Rating"]].values
    
    for epoch in range(epochs):

        erroraccum = 0
        count = 0
        #np.random.shuffle(SList)

        for user, item, rating in SList:
            i = user_index[user]
            j = item_index[item]
            error = rating - np.dot(P[i, :], Q[j, :].T)

            for l in range(k):
                Pnew = error * Q[j,l]
                Qnew = error * P[i,l]
                P[i,l] += alpha * (Pnew - lamb * P[i,l])
                Q[j,l] += alpha * (Qnew - lamb * Q[j,l])
            
            erroraccum += error**2
            count +=1
        
        mse = erroraccum / count
        print("Epoch: " + str(epoch+1) + " Error: " + str(np.sqrt(mse)))

    
    return P, Q, user_index, item_index


In [ ]:
#Execução do algoritmo
#Executa o algoritmo SGD para os ratings de treino
P, Q, user_index, item_index = SGD(df_ratings)

Epoch: 1 Error: 1.4583975837562
Epoch: 2 Error: 1.2621144276082301
Epoch: 3 Error: 1.138456201361276
Epoch: 4 Error: 1.043195966984619
Epoch: 5 Error: 0.9648282598790486
Epoch: 6 Error: 0.8979523248513906
Epoch: 7 Error: 0.8394939974049314
Epoch: 8 Error: 0.7875115111015171
Epoch: 9 Error: 0.7406952133006728
Epoch: 10 Error: 0.69812076711483


In [ ]:
#Calcula os ratings previstos para os ratings de teste
df_targets = predict_all_ratings(df_targets, P, Q, user_index, item_index)

In [ ]:
#Juntar UserId e ItemId e Rating em uma coluna para gerar o arquivo de submissão final da forma UserId:ItemId,Rating
df_targets["UserId:ItemId"] = df_targets["UserId"].astype(str) + ":" + df_targets["ItemId"].astype(str)
df_targets = df_targets[["UserId:ItemId", "Rating"]]


In [ ]:
#Gerar arquivo csv para submissão com UserId:ItemId,Rating na primeira linha e os ratings previstos nas linhas seguintes

df_targets.to_csv("submission.csv", index=False, header=False)

#Adicionar UserId:ItemId,Rating na primeira linha do arquivo submission.csv
with open('submission.csv', 'r') as original: data = original.read()
with open('submission.csv', 'w') as modified: modified.write("UserId:ItemId,Rating\n" + data)


In [ ]:
#importar o df resultados.csv para um dataframe e ordenar por numberNodes
df_results = pd.read_csv('resultados.csv', sep=',', header=0)
df_results.sort_values(by=['numberNodes'], inplace=True)

df_results.head()